In [1]:
from huggingface_hub import hf_hub_download
hf_hub_download(repo_id="BlinkDL/rwkv-4-pile-14b", filename="RWKV-4-Pile-14B-20230213-8019.pth", cache_dir="/home/kyle/HF-MODEL/rwkv-4-pile-14b")

/home/kyle/miniconda3/envs/kyle-elk-rwkv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'/home/kyle/HF-MODEL/rwkv-4-pile-14b/models--BlinkDL--rwkv-4-pile-14b/snapshots/939b6851f96122b7b49bd00d446b3b49481214dd/RWKV-4-Pile-14B-20230213-8019.pth'

In [23]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5"
os.environ["RWKV_JIT_ON"] = '1'
os.environ["RWKV_CUDA_ON"] = '0' # if '1' then use CUDA kernel for seq mode (much faster)
from rwkv.model import RWKV                         # pip install rwkv
model = RWKV(model='/home/kyle/HF-MODEL/rwkv-4-pile-1b5/models--BlinkDL--rwkv-4-pile-1b5/snapshots/6ea995eaa87a17af560c9b41ce1a3d92355c5a49/RWKV-4-Pile-1B5-20220903-8040.pth', strategy='cuda fp16')

out, state = model.forward([187, 510, 1563, 310, 247], None)   # use 20B_tokenizer.json
print(out.detach().cpu().numpy())                   # get logits
out, state = model.forward([187, 510], None)
out, state = model.forward([1563], state)           # RNN has state (use deepcopy if you want to clone it)
out, state = model.forward([310, 247], state)
print(out.detach().cpu().numpy())                   # same result as above

RWKV_JIT_ON 1 RWKV_CUDA_ON 0 RESCALE_LAYER 6

Loading /home/kyle/HF-MODEL/rwkv-4-pile-1b5/models--BlinkDL--rwkv-4-pile-1b5/snapshots/6ea995eaa87a17af560c9b41ce1a3d92355c5a49/RWKV-4-Pile-1B5-20220903-8040.pth ...
Strategy: (total 24+1=25 layers)
* cuda [float16, float16], store 25 layers
0-cuda-float16-float16 1-cuda-float16-float16 2-cuda-float16-float16 3-cuda-float16-float16 4-cuda-float16-float16 5-cuda-float16-float16 6-cuda-float16-float16 7-cuda-float16-float16 8-cuda-float16-float16 9-cuda-float16-float16 10-cuda-float16-float16 11-cuda-float16-float16 12-cuda-float16-float16 13-cuda-float16-float16 14-cuda-float16-float16 15-cuda-float16-float16 16-cuda-float16-float16 17-cuda-float16-float16 18-cuda-float16-float16 19-cuda-float16-float16 20-cuda-float16-float16 21-cuda-float16-float16 22-cuda-float16-float16 23-cuda-float16-float16 24-cuda-float16-float16 
emb.weight                        f16      cpu  50277  2048 
blocks.0.ln1.weight               f16   cuda:0   2048       

In [3]:
from transformers import AutoTokenizer, GPT2TokenizerFast
tokenizer = GPT2TokenizerFast(tokenizer_file='20B_tokenizer.json')
text = "The following movie review expresses what sentiment? George P. Cosmatos\' \"Rambo: First Blood Part II\" is pure wish-fulfillment. The United States clearly didn\'t win the war in Vietnam. They caused damage to this country beyond the imaginable and this movie continues the fairy story of the oh-so innocent soldiers. The only bad guys were the leaders of the nation, who made this war happen. The character of Rambo is perfect to notice this. He is extremely patriotic, bemoans that US-Americans didn\'t appreciate and celebrate the achievements of the single soldier, but has nothing but distrust for leading officers and politicians. Like every film that defends the war (e.g. \"We Were Soldiers\") also this one avoids the need to give a comprehensible reason for the engagement in South Asia. And for that matter also the reason for every single US-American soldier that was there. Instead, Rambo gets to take revenge for the wounds of a whole nation. It would have been better to work on how to deal with the memories, rather than suppressing them. \"Do we get to win this time?\" Yes, you do.\n\nnegative"

In [4]:
# Return as tensor
tokens = tokenizer(text, return_tensors="pt")
out, state = model.forward(tokens["input_ids"], None)
state[-1].shape # torch.Size([241, 2048]) w/ tensor

torch.Size([241, 2048])

In [5]:
# Return as list
tokens = tokenizer(text)
out, state = model.forward(tokens["input_ids"], None)
state[-1].shape # torch.Size([2048]) w/o tensor

torch.Size([2048])

In [6]:
len(
state)

120

In [7]:
len(state)

120

In [8]:
tokenizer.decode(out.argmax())

'\n'

In [9]:
state[-1]

tensor([ 0.3020, -0.3469, -0.0307,  ...,  0.1010,  0.0585, -0.1140],
       device='cuda:0', dtype=torch.float16)

In [10]:
state[-1].shape

torch.Size([2048])

In [11]:
len(state)

120

In [12]:
out, state = model.forward([187, 510, 1563, 310, 247], None)   # use 20B_tokenizer.json
print(out.detach().cpu().numpy())                   # get logits
out, state = model.forward([187, 510], None)
out, state = model.forward([1563], state)           # RNN has state (use deepcopy if you want to clone it)
out, state = model.forward([310, 247], state)

[ -8.828125  -21.828125   -9.359375  ...  -7.7070312  -4.875
  -1.9785156]


In [13]:
from rwkv.utils import PIPELINE, PIPELINE_ARGS
pipeline = PIPELINE(model, "20B_tokenizer.json") 
ctx = "\nIn a shocking finding, scientist discovered a herd of dragons living in a remote, previously unexplored valley, in Tibet. Even more surprising to the researchers was the fact that the dragons spoke perfect Chinese."
print(ctx, end='')

def my_print(s):
    print(s, end='', flush=True)

args = PIPELINE_ARGS(temperature = 1.0, top_p = 0.7, top_k = 100, # top_k = 0 then ignore
                     alpha_frequency = 0.25,
                     alpha_presence = 0.25,
                     token_ban = [0], # ban the generation of some tokens
                     token_stop = [], # stop generation whenever you see any token here
                     chunk_len = 256) # split input into chunks to save VRAM (shorter -> slower)

pipeline.generate(ctx, token_count=200, args=args, callback=my_print)


In a shocking finding, scientist discovered a herd of dragons living in a remote, previously unexplored valley, in Tibet. Even more surprising to the researchers was the fact that the dragons spoke perfect Chinese.

The research was conducted by researchers from the Nanjing Institute of Geology and Paleontology, the School of Chinese Academy of Sciences and Sichuan University.

The finding is an important breakthrough in studying dragon evolution, said Jianfeng Zhang, a scientist at the Nanjing Institute of Geology and Paleontology and the School of Chinese Academy of Sciences.

Dragon lore tells us that dragons live in a hidden valley near Mount Kailash in central Tibet, which is an enormous body of water between mountains. The valleys contain fertile soils and are home to large herds of woolly mammoths, which left behind traces on the valley floor. Draculas also live there, but are mostly vegetarian.

One year, scientists came across five female dragons who were lactating. At that t

'\n\nThe research was conducted by researchers from the Nanjing Institute of Geology and Paleontology, the School of Chinese Academy of Sciences and Sichuan University.\n\nThe finding is an important breakthrough in studying dragon evolution, said Jianfeng Zhang, a scientist at the Nanjing Institute of Geology and Paleontology and the School of Chinese Academy of Sciences.\n\nDragon lore tells us that dragons live in a hidden valley near Mount Kailash in central Tibet, which is an enormous body of water between mountains. The valleys contain fertile soils and are home to large herds of woolly mammoths, which left behind traces on the valley floor. Draculas also live there, but are mostly vegetarian.\n\nOne year, scientists came across five female dragons who were lactating. At that time, dragon eggs were very rare in the region. The team found out that some years before, dragons had been breeding with some other species. That’'

In [14]:
pipeline.encode(ctx) == GPT2TokenizerFast(tokenizer_file='20B_tokenizer.json').encode(ctx)

True

In [45]:
import os
from rwkv_hf import RWKVModel
from transformers import GPT2TokenizerFast

os.environ["CUDA_VISIBLE_DEVICES"] = "4"

tokenizer = GPT2TokenizerFast(tokenizer_file='20B_tokenizer.json')
bad_review_text = 'George P. Cosmatos\' "Rambo: First Blood Part II" is pure wish-fulfillment. The United States clearly didn\'t win the war in Vietnam. They caused damage to this country beyond the imaginable and this movie continues the fairy story of the oh-so innocent soldiers. The only bad guys were the leaders of the nation, who made this war happen. The character of Rambo is perfect to notice this. He is extremely patriotic, bemoans that US-Americans didn\'t appreciate and celebrate the achievements of the single soldier, but has nothing but distrust for leading officers and politicians. Like every film that defends the war (e.g. "We Were Soldiers") also this one avoids the need to give a comprehensible reason for the engagement in South Asia. And for that matter also the reason for every single US-American soldier that was there. Instead, Rambo gets to take revenge for the wounds of a whole nation. It would have been better to work on how to deal with the memories, rather than suppressing them. "Do we get to win this time?" Yes, you do.\n\nDid the reviewer find this movie good or bad? bad'
good_review_text = 'George P. Cosmatos\' "Rambo: First Blood Part II" is pure wish-fulfillment. The United States clearly didn\'t win the war in Vietnam. They caused damage to this country beyond the imaginable and this movie continues the fairy story of the oh-so innocent soldiers. The only bad guys were the leaders of the nation, who made this war happen. The character of Rambo is perfect to notice this. He is extremely patriotic, bemoans that US-Americans didn\'t appreciate and celebrate the achievements of the single soldier, but has nothing but distrust for leading officers and politicians. Like every film that defends the war (e.g. "We Were Soldiers") also this one avoids the need to give a comprehensible reason for the engagement in South Asia. And for that matter also the reason for every single US-American soldier that was there. Instead, Rambo gets to take revenge for the wounds of a whole nation. It would have been better to work on how to deal with the memories, rather than suppressing them. "Do we get to win this time?" Yes, you do.\n\nDid the reviewer find this movie good or bad? good'
model = RWKVModel()

RWKV_JIT_ON 1 RWKV_CUDA_ON 0 RESCALE_LAYER 0

Loading /home/kyle/HF-MODEL/rwkv-4-pile-1b5/models--BlinkDL--rwkv-4-pile-1b5/snapshots/6ea995eaa87a17af560c9b41ce1a3d92355c5a49/RWKV-4-Pile-1B5-20220903-8040.pth ...
Strategy: (total 24+1=25 layers)
* cuda [bfloat16, bfloat16], store 25 layers
0-cuda-bfloat16-bfloat16 1-cuda-bfloat16-bfloat16 2-cuda-bfloat16-bfloat16 3-cuda-bfloat16-bfloat16 4-cuda-bfloat16-bfloat16 5-cuda-bfloat16-bfloat16 6-cuda-bfloat16-bfloat16 7-cuda-bfloat16-bfloat16 8-cuda-bfloat16-bfloat16 9-cuda-bfloat16-bfloat16 10-cuda-bfloat16-bfloat16 11-cuda-bfloat16-bfloat16 12-cuda-bfloat16-bfloat16 13-cuda-bfloat16-bfloat16 14-cuda-bfloat16-bfloat16 15-cuda-bfloat16-bfloat16 16-cuda-bfloat16-bfloat16 17-cuda-bfloat16-bfloat16 18-cuda-bfloat16-bfloat16 19-cuda-bfloat16-bfloat16 20-cuda-bfloat16-bfloat16 21-cuda-bfloat16-bfloat16 22-cuda-bfloat16-bfloat16 23-cuda-bfloat16-bfloat16 24-cuda-bfloat16-bfloat16 
emb.weight                       bf16      cpu  50277  2048 
blocks.0

In [46]:
tokens = tokenizer.encode(good_review_text, return_tensors="pt")
print(f"Input IDs shape = {tokens.shape}")
states = model(tokens)
tensor_final_state = states[-1]
tensor_final_state.shape

Input IDs shape = torch.Size([1, 243])


torch.Size([243, 2048])

In [52]:
tensor_final_state[-1]

tensor([ 0.3164, -0.3691,  0.0165,  ..., -0.0081,  0.0175, -0.0771],
       device='cuda:0', dtype=torch.bfloat16)

In [58]:
tokens = tokenizer.encode(bad_review_text, return_tensors="pt")
print(f"Input IDs shape = {tokens.shape}")

states = model(tokens)
tensor_final_state = states[-1]
tensor_final_state.shape

Input IDs shape = torch.Size([1, 243])


torch.Size([243, 2048])

In [59]:
tokens = tokenizer.encode(bad_review_text, return_tensors="pt")
print(f"Input IDs shape = {tokens.shape}")

states = model(tokens[0])
tensor_final_state = states[-1]
tensor_final_state.shape

Input IDs shape = torch.Size([1, 243])


torch.Size([2048])